In [19]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import string ,random
import numpy as np
import torch.nn.functional as F
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from collections import Counter
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, BertTokenizer
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

base_folder = "D:\\haas-mfe-g24-cg\\AFP_test\\"


cuda


In [20]:
def build_dictionary(dictionary_file_location):
    text_file = open(dictionary_file_location,"r")
    full_dictionary = text_file.read().splitlines()
    text_file.close()
    return full_dictionary

def adjust_temperature(logits, temperature=1.0):
    # Apply temperature scaling to logits
    scaled_logits = logits / temperature

    # Apply softmax to obtain probabilities
    probabilities = F.softmax(scaled_logits, dim=-1)

    return probabilities

def bert_guess_func(curr_word, guessed_letters ,tokenizer, model):
    
    curr_word2 = " ".join(list(curr_word))
    tokenized_sequence = tokenizer.tokenize(curr_word2.replace('_', '[MASK]'))
    # print(tokenized_sequence)

    input_ids = tokenizer.convert_tokens_to_ids(tokenized_sequence)
    input_ids = torch.tensor([input_ids])
    with torch.no_grad():
        outputs = model(input_ids)
    predictions = outputs.logits[0, -1, :]
    if len(guessed_letters) < 2:
        temperature   = 1.0
        probabilities = adjust_temperature(predictions, temperature)
    else:
        temperature   = 1.0
        probabilities = adjust_temperature(predictions, temperature)
    
    alphabet_characters = set(string.ascii_lowercase)
    filtered_probs = [prob.item() if token in alphabet_characters and token not in guessed_letters and token not in curr_word else float('-inf') for prob, token in zip(probabilities, tokenizer.convert_ids_to_tokens(range(len(probabilities))))]
    predicted_index = filtered_probs.index(max(filtered_probs))
    predicted_letter = tokenizer.convert_ids_to_tokens([predicted_index])

    return predicted_letter

def bert_guess_func2(curr_word, guessed_letters ,tokenizer, model):
    
    # curr_word2 = " ".join(list(curr_word))
    tokenized_sequence = tokenizer.tokenize(curr_word.replace('_', '[MASK]'))
    # print(tokenized_sequence)

    input_ids = tokenizer.convert_tokens_to_ids(tokenized_sequence)
    input_ids = torch.tensor([input_ids])
    with torch.no_grad():
        outputs = model(input_ids)
    logits = outputs.logits[0, -1, :]
    probabilities = F.softmax(logits, dim=-1)
    
    filtered_probs = [
        prob.item() if token.isalpha() and token.isascii() else 0
        for prob, token in zip(probabilities, tokenizer.convert_ids_to_tokens(range(len(probabilities))))
    ]

    non_zero_tokens = [token for token, prob in zip(tokenizer.convert_ids_to_tokens(range(len(probabilities))), filtered_probs) if prob != 0]
    all_text = ' '.join(non_zero_tokens)
    alphabet_counts = Counter(char.lower() for char in all_text if char.isalpha())

    for letter in guessed_letters:
        del alphabet_counts[letter]

    most_common_alphabet, most_common_count = alphabet_counts.most_common(1)[0]

    return most_common_alphabet

def bert_guess_func_train(curr_word, filled_alphabets ,tokenizer, model):
    
    curr_word2 = " ".join(list(curr_word))
    tokenized_sequence = tokenizer.tokenize(curr_word2.replace('_', '[MASK]'))
    tokenized_sequence = tokenized_sequence[:max_sequence_length] + ['[PAD]'] * (max_sequence_length - len(tokenized_sequence))
    
    # print(tokenized_sequence)

    input_ids = tokenizer.convert_tokens_to_ids(tokenized_sequence)
    
    input_ids = torch.tensor([input_ids])
    input_ids = input_ids.to(device)
    
    with torch.no_grad():
        outputs = model(input_ids)
        
    predictions = outputs.logits[0, :]
    probabilities = F.softmax(predictions, dim=-1)
    
    # if len(guessed_letters) < 2:
    #     temperature   = 1.0
    #     probabilities = adjust_temperature(predictions, temperature)
    # else:
    #     temperature   = 1.0
    #     probabilities = adjust_temperature(predictions, temperature)

    dict_alphas  = set([chr(word + ord('a')) for word in np.arange(26) ])
    valid_alphas = list(dict_alphas.difference(filled_alphabets))
    valid_idxs   = [ord(x) - ord('a') for x in valid_alphas]
    #max_idx      = valid_idxs[np.argmax(probabilities[valid_idxs])]
    max_idx = valid_idxs[torch.argmax(probabilities[valid_idxs])]

    alphabet      = chr(max_idx + ord('a'))
    

    return alphabet


In [21]:

class data_maker:
    
    def __init__(self,full_dictionary ,TOTX , feature_type ,NUM_FEATURES):
        
        self.full_dictionary = full_dictionary
        self.TOTX  = TOTX        
        self.NUM_FEATURES = NUM_FEATURES
        self.feature_type = feature_type

    def label_maker(self,curr_word ,full_word):
            
        char_array = np.zeros(26)
        
        for i,letter in enumerate(curr_word):
            if letter == '_':
                char_array[ord(full_word[i]) - ord('a')] = 1
        
        return char_array

                    
    def word_splitter(self,full_word):
        
        tot_len = len(full_word)
        remaining_letters = random.randint(0, tot_len - 1)
        chosen_letters_indices = random.sample(range(tot_len), remaining_letters)
        curr_word = [full_word[i] if i in chosen_letters_indices else '_' for i in range(tot_len)]
        curr_word = ''.join(curr_word)
            
        return curr_word , full_word

    def word_splitter2(self,full_word):
        
        tot_len = len(full_word)
        remaining_letters = random.randint(0, tot_len - 1)
        chosen_letters_indices = random.sample(range(tot_len), remaining_letters)
        curr_word = [full_word[i] if i in chosen_letters_indices else '_' for i in range(tot_len)]
        curr_word = ''.join(curr_word)

        non_curr_word = [full_word[i] if i not in chosen_letters_indices else '' for i in range(tot_len)]
        non_curr_word = ''.join(non_curr_word)
            
        return curr_word , full_word ,non_curr_word

        
    def update_word(self,curr_word, full_word, alphabet):
        
        updated_word = list(curr_word)
        
        for i, char in enumerate(curr_word):
            if curr_word[i] == '_' and full_word[i] == alphabet:
                updated_word[i] = alphabet
                
        return ''.join(updated_word)
            
    def check_strategy(self ,curr_word ,full_word,NUM_TRIES ,tokenizer,model):
        
        tot_len          = len(curr_word)        
        filled_len       = len([char for char in curr_word if char != '_'])
        empty_len        = tot_len - filled_len
        filled_alphabets = {char for char in curr_word if char.isalpha()}
        
        while NUM_TRIES>0:
            
            # alphabet = bert_guess_func(curr_word ,filled_alphabets ,tokenizer,model)[0]
            # alphabet = bert_guess_func2(curr_word ,filled_alphabets ,tokenizer,model)[0]
            alphabet = bert_guess_func_train(curr_word ,filled_alphabets ,tokenizer,model)[0]
            
            updated_word = list(curr_word)
            check = 0
            for i, char in enumerate(curr_word):
                if curr_word[i] == '_' and full_word[i] == alphabet:
                    check = 1
                    updated_word[i] = alphabet
    
            curr_word =  ''.join(updated_word)
            if check == 0:
                NUM_TRIES = NUM_TRIES - 1
                        
            print(NUM_TRIES," " ,alphabet, " " , curr_word, " " ,filled_alphabets)
            filled_alphabets.add(alphabet)
            # print(filled_alphabets)
        
            filled_len       = len([char for char in curr_word if char != '_'])
            if filled_len == tot_len:
                return 1
            
        return 0

    def test(self ,NUM_TRIES , NUM_WORDS ,tokenizer,model):
        
        tot_ans = 0
        for i in np.arange(NUM_WORDS):
            
            NUM_TRIES = 6        
            full_word = random.choice(self.full_dictionary)
            # full_word = 'naieve'
        
            updated_word = list()
            for i, char in enumerate(full_word):
                updated_word.append('_')
        
            curr_word =  ''.join(updated_word)  # Convert the list back to a string    
                        
            print(full_word)
            ans = self.check_strategy(curr_word,full_word,NUM_TRIES ,tokenizer,model)
            print(ans)
            tot_ans += ans
        print("ratio " ,tot_ans/NUM_WORDS)
                
    def get_masked_words(self,NUM_WORDS):

        masked_words = []
        full_words   = []
        non_curr_words   = []
        
        label_array   = np.zeros(shape = (NUM_WORDS ,26))
            
        for i in np.arange(NUM_WORDS):
            full_word             = random.choice(self.full_dictionary)
            curr_word , full_word ,non_curr_word = self.word_splitter2(full_word)

            masked_words.append(curr_word)
            full_words.append(full_word)
            non_curr_words.append(non_curr_word)
            labels                = self.label_maker(curr_word ,full_word)
            label_array[i,:]      = labels

        return masked_words , label_array ,full_words



In [ ]:
TREXQUANT - //9 (4) ;  _ _ E X Q U A _ _ ;   -> T R N T

In [22]:

full_dictionary = build_dictionary(base_folder + "words_250000_train.txt")

DICTIONARY_SIZE = 1250000
FEATURE_TYPE    = 0
NUM_FEATURES    = 30 # 706
num_classes     = 26
max_sequence_length = 27
BATCH_SIZE     = 256

curr_word = "rock__ar"

dm = data_maker(full_dictionary ,DICTIONARY_SIZE , FEATURE_TYPE , NUM_FEATURES)



In [23]:
train_words ,train_label_words ,train_full_words = dm.get_masked_words(DICTIONARY_SIZE)

In [24]:
valid_words ,valid_label_words ,valid_full_words = dm.get_masked_words(int(DICTIONARY_SIZE/10))

In [25]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)
model.to(device)  # Move the model to GPU

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [26]:
# Tokenize and encode the data
tokenized_inputs = []

for ii in np.arange(len(train_full_words)):
    
    partial_word        = train_words[ii]
    partial_word        = " ".join(list(partial_word))

    tokenized_sequence = tokenizer.tokenize(partial_word.replace('_', '[MASK]'))
    tokenized_sequence = tokenized_sequence[:max_sequence_length] + ['[PAD]'] * (max_sequence_length - len(tokenized_sequence))

    input_ids = tokenizer.convert_tokens_to_ids(tokenized_sequence)
    tokenized_inputs.append(input_ids)


In [27]:
len(tokenized_inputs[0])

27

In [28]:
# Convert to PyTorch tensors
input_ids_tensor = torch.tensor(tokenized_inputs)
labels_tensor = torch.tensor(train_label_words)

input_ids_tensor = input_ids_tensor.to(device)
labels_tensor = labels_tensor.to(device)

# Create a DataLoader
train_dataset = TensorDataset(input_ids_tensor, labels_tensor)
train_dataloader = DataLoader(train_dataset, batch_size= BATCH_SIZE, shuffle=True)


In [29]:
# Tokenize and encode the data
valid_tokenized_inputs = []


for ii in np.arange(len(valid_full_words)):
    
    valid_partial_word = valid_words[ii]
    valid_partial_word = " ".join(list(valid_partial_word))
        
    valid_tokenized_sequence = tokenizer.tokenize(valid_partial_word.replace('_', '[MASK]'))
    valid_tokenized_sequence = valid_tokenized_sequence[:max_sequence_length] + ['[PAD]'] * (max_sequence_length - len(valid_tokenized_sequence))

    valid_input_ids = tokenizer.convert_tokens_to_ids(valid_tokenized_sequence)
    valid_tokenized_inputs.append(valid_input_ids)

    

In [30]:
len(valid_tokenized_inputs[0])

27

In [31]:
# Convert to PyTorch tensors
valid_input_ids_tensor = torch.tensor(valid_tokenized_inputs)
valid_labels_tensor    = torch.tensor(valid_label_words)

valid_input_ids_tensor = valid_input_ids_tensor.to(device)
valid_labels_tensor    = valid_labels_tensor.to(device)

# Create a DataLoader
valid_dataset    = TensorDataset(valid_input_ids_tensor, valid_labels_tensor)
valid_dataloader = DataLoader(valid_dataset, batch_size= BATCH_SIZE)

In [32]:
# Convert to PyTorch tensors
valid_input_ids_tensor = torch.tensor(valid_tokenized_inputs)
valid_labels_tensor    = torch.tensor(valid_label_words)

valid_input_ids_tensor = valid_input_ids_tensor.to(device)
valid_labels_tensor    = valid_labels_tensor.to(device)

# Create a DataLoader
valid_dataset    = TensorDataset(valid_input_ids_tensor, valid_labels_tensor)
valid_dataloader = DataLoader(valid_dataset, batch_size=128, shuffle=True)


In [33]:
# Define loss function and optimizer

optimizer = Adam(model.parameters(), lr= 1e-5)  # You may need to adjust the learning rate

# Training loop
num_epochs = 25  # Adjust as needed
criterion = BCEWithLogitsLoss()


In [34]:

# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0.0

#     for step, batch in enumerate(train_dataloader):
#         inputs, labels = batch

#         # Forward pass
#         outputs = model(inputs)
#         logits = outputs.logits

#         # Compute loss
#         loss = criterion(logits, labels)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#         # Print loss at each step
#         if step%200 == 0:
#             print(f'Epoch {epoch + 1}/{num_epochs}, Step {step + 1}/{len(train_dataloader)}, Loss: {loss.item()}')


#     average_loss = total_loss / len(train_dataloader)
#     print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}')



In [35]:

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for step, batch in enumerate(train_dataloader):
        inputs, labels = batch

        # Forward pass
        outputs = model(inputs)
        logits = outputs.logits

        # Compute loss
        loss = criterion(logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if step%500 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Step {step + 1}/{len(train_dataloader)}, Loss: {loss.item()}')


    average_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}')

    model.eval()
    total_valid_loss = 0.0

    with torch.no_grad():
        for step, valid_batch in enumerate(valid_dataloader):
            inputs, labels = valid_batch

            # Forward pass
            outputs = model(inputs)
            logits = outputs.logits

            # Compute validation loss
            valid_loss = criterion(logits, labels)

            total_valid_loss += valid_loss.item()

    average_valid_loss = total_valid_loss / len(valid_dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {average_valid_loss}')


Epoch 1/25, Step 1/4883, Loss: 0.7177793437850387
Epoch 1/25, Step 501/4883, Loss: 0.3568402794572614
Epoch 1/25, Step 1001/4883, Loss: 0.355065682793443
Epoch 1/25, Step 1501/4883, Loss: 0.3376701051030033
Epoch 1/25, Step 2001/4883, Loss: 0.3474476591845814
Epoch 1/25, Step 2501/4883, Loss: 0.3449850686625863
Epoch 1/25, Step 3001/4883, Loss: 0.34170083955258146
Epoch 1/25, Step 3501/4883, Loss: 0.3362330487032768
Epoch 1/25, Step 4001/4883, Loss: 0.33802653542481775
Epoch 1/25, Step 4501/4883, Loss: 0.34021102339025117
Epoch 1/25, Loss: 0.35237229269536113
Epoch 1/25, Validation Loss: 0.33573477498901866
Epoch 2/25, Step 1/4883, Loss: 0.34823493275154344
Epoch 2/25, Step 501/4883, Loss: 0.3460032402649019
Epoch 2/25, Step 1001/4883, Loss: 0.3256097293123223
Epoch 2/25, Step 1501/4883, Loss: 0.3309611867402877
Epoch 2/25, Step 2001/4883, Loss: 0.32546571809580904
Epoch 2/25, Step 2501/4883, Loss: 0.33843262657168577
Epoch 2/25, Step 3001/4883, Loss: 0.31871968501043896
Epoch 2/25, St

Epoch 14/25, Step 1501/4883, Loss: 0.3139125786542357
Epoch 14/25, Step 2001/4883, Loss: 0.29301333950511155
Epoch 14/25, Step 2501/4883, Loss: 0.3060549842658495
Epoch 14/25, Step 3001/4883, Loss: 0.31780039874236615
Epoch 14/25, Step 3501/4883, Loss: 0.3115493106576729
Epoch 14/25, Step 4001/4883, Loss: 0.3123697407749326
Epoch 14/25, Step 4501/4883, Loss: 0.301934829214964
Epoch 14/25, Loss: 0.3014490343412114
Epoch 14/25, Validation Loss: 0.3104515013252089
Epoch 15/25, Step 1/4883, Loss: 0.2768779799469261
Epoch 15/25, Step 501/4883, Loss: 0.294471550215225
Epoch 15/25, Step 1001/4883, Loss: 0.30627482488987484
Epoch 15/25, Step 1501/4883, Loss: 0.31800117339391487
Epoch 15/25, Step 2001/4883, Loss: 0.31013772068545303
Epoch 15/25, Step 2501/4883, Loss: 0.28787079707965973
Epoch 15/25, Step 3001/4883, Loss: 0.2980379891793865
Epoch 15/25, Step 3501/4883, Loss: 0.2880748875768626
Epoch 15/25, Step 4001/4883, Loss: 0.3104019440073326
Epoch 15/25, Step 4501/4883, Loss: 0.293888677140

In [36]:
# Save the trained model
import datetime
now = datetime.datetime.now()

date_time_str = now.strftime("%Y%m%d_%H%M%S")    
model.save_pretrained(f"fine_tuned_bert_model{date_time_str}.h5")


In [41]:
NUM_TRIES       = 6
NUM_WORDS       = 1000

In [42]:

dm.test(NUM_TRIES,NUM_WORDS,tokenizer,model)


outmoding
5   e   _________   set()
5   i   ______i__   {'e'}
5   n   ______in_   {'i', 'e'}
5   g   ______ing   {'i', 'n', 'e'}
4   r   ______ing   {'i', 'g', 'n', 'e'}
3   a   ______ing   {'g', 'i', 'r', 'e', 'n'}
3   t   __t___ing   {'g', 'i', 'r', 'a', 'e', 'n'}
3   u   _ut___ing   {'g', 'i', 'r', 'a', 'e', 't', 'n'}
3   o   out_o_ing   {'g', 'u', 'i', 'r', 'a', 'e', 't', 'n'}
2   l   out_o_ing   {'g', 'u', 'i', 'r', 'a', 'e', 't', 'n', 'o'}
1   w   out_o_ing   {'g', 'u', 'i', 'r', 'a', 'e', 'l', 't', 'n', 'o'}
0   p   out_o_ing   {'g', 'u', 'w', 'i', 'r', 'a', 'e', 'l', 't', 'n', 'o'}
0
vowelled
6   e   ___e__e_   set()
5   r   ___e__e_   {'e'}
5   d   ___e__ed   {'r', 'e'}
4   n   ___e__ed   {'r', 'd', 'e'}
3   t   ___e__ed   {'r', 'n', 'd', 'e'}
2   u   ___e__ed   {'r', 'e', 't', 'n', 'd'}
1   i   ___e__ed   {'u', 'r', 'e', 't', 'n', 'd'}
0   s   ___e__ed   {'u', 'i', 'r', 'e', 't', 'n', 'd'}
0
teratomata
6   e   _e________   set()
6   r   _er_______   {'e'}
5   i   _er_______  

1   n   _o___o_es   {'i', 'r', 's', 'a', 'e', 'o', 'd'}
1   l   _o_l_o_es   {'i', 'r', 's', 'a', 'e', 'o', 'n', 'd'}
0   b   _o_l_o_es   {'i', 'r', 's', 'a', 'e', 'o', 'l', 'n', 'd'}
0
entering
6   e   e__e____   set()
6   t   e_te____   {'e'}
6   r   e_ter___   {'t', 'e'}
6   n   enter_n_   {'t', 'r', 'e'}
6   i   enterin_   {'t', 'r', 'n', 'e'}
6   g   entering   {'i', 'r', 'e', 't', 'n'}
1
bengaline
6   e   _e______e   set()
5   r   _e______e   {'e'}
5   i   _e____i_e   {'r', 'e'}
4   t   _e____i_e   {'i', 'r', 'e'}
4   n   _en___ine   {'i', 'r', 't', 'e'}
3   c   _en___ine   {'i', 'r', 'e', 't', 'n'}
3   l   _en__line   {'i', 'r', 'e', 'c', 't', 'n'}
3   b   ben__line   {'i', 'r', 'e', 'c', 'l', 't', 'n'}
2   z   ben__line   {'b', 'i', 'r', 'e', 'c', 'l', 't', 'n'}
2   a   ben_aline   {'z', 'b', 'i', 'r', 'e', 'c', 'l', 't', 'n'}
1   v   ben_aline   {'z', 'b', 'i', 'r', 'a', 'e', 'c', 'l', 't', 'n'}
0   d   ben_aline   {'z', 'b', 'i', 'r', 'v', 'a', 'e', 'c', 'l', 't', 'n'}
0
prosa

4   t   __as_ati_   {'i', 's', 'a', 'e', 'c'}
4   m   __asmati_   {'i', 's', 'a', 'e', 'c', 't'}
4   p   p_asmati_   {'m', 'i', 's', 'a', 'e', 'c', 't'}
3   l   p_asmati_   {'m', 'i', 'p', 's', 'a', 'e', 'c', 't'}
3   h   phasmati_   {'m', 'i', 'p', 's', 'a', 'e', 'c', 'l', 't'}
2   r   phasmati_   {'m', 'h', 'i', 'p', 's', 'a', 'e', 'c', 'l', 't'}
1   n   phasmati_   {'m', 'h', 'i', 'p', 's', 'a', 'r', 'e', 'c', 'l', 't'}
1   d   phasmatid   {'m', 'h', 'i', 'p', 's', 'a', 'r', 'e', 'c', 'l', 't', 'n'}
1
byon
5   a   ____   set()
4   e   ____   {'a'}
3   s   ____   {'a', 'e'}
3   o   __o_   {'s', 'a', 'e'}
2   l   __o_   {'s', 'a', 'o', 'e'}
1   r   __o_   {'s', 'a', 'e', 'l', 'o'}
1   n   __on   {'r', 's', 'a', 'e', 'l', 'o'}
0   p   __on   {'r', 's', 'a', 'e', 'l', 'n', 'o'}
0
manifestations
6   i   ___i______i___   set()
6   a   _a_i____a_i___   {'i'}
6   n   _ani____a_i_n_   {'i', 'a'}
6   s   _ani__s_a_i_ns   {'i', 'a', 'n'}
6   o   _ani__s_a_ions   {'i', 's', 'a', 'n'}
6   t   _a

5   n   __n___i_n   {'i', 'e'}
4   o   __n___i_n   {'i', 'n', 'e'}
4   a   _an_a_ian   {'i', 'n', 'o', 'e'}
4   t   tanta_ian   {'i', 'a', 'e', 'n', 'o'}
3   r   tanta_ian   {'i', 'a', 'e', 't', 'n', 'o'}
3   l   tantalian   {'i', 'r', 'a', 'e', 't', 'n', 'o'}
1
metanitrophenol
6   i   _____i_________   set()
6   n   ____ni______n__   {'i'}
6   o   ____ni__o___no_   {'i', 'n'}
6   e   _e__ni__o__eno_   {'i', 'n', 'o'}
6   h   _e__ni__o_heno_   {'i', 'n', 'o', 'e'}
6   t   _et_nit_o_heno_   {'h', 'i', 'e', 'n', 'o'}
6   r   _et_nitro_heno_   {'h', 'i', 'e', 't', 'n', 'o'}
6   p   _et_nitropheno_   {'h', 'i', 'r', 'e', 't', 'n', 'o'}
6   m   met_nitropheno_   {'h', 'i', 'r', 'p', 'e', 't', 'n', 'o'}
6   a   metanitropheno_   {'m', 'h', 'i', 'r', 'p', 'e', 't', 'n', 'o'}
6   l   metanitrophenol   {'m', 'h', 'i', 'r', 'p', 'a', 'e', 't', 'n', 'o'}
1
tudoresque
6   e   _____e___e   set()
5   i   _____e___e   {'e'}
5   r   ____re___e   {'i', 'e'}
5   t   t___re___e   {'i', 'r', 'e'}
4   a   

0   d   pro_es_er___   {'g', 'p', 'r', 's', 'i', 'e', 'f', 't', 'n', 'o'}
0
croatan
5   e   _______   set()
5   a   ___a_a_   {'e'}
5   n   ___a_an   {'a', 'e'}
4   m   ___a_an   {'a', 'n', 'e'}
4   r   _r_a_an   {'m', 'a', 'n', 'e'}
3   i   _r_a_an   {'m', 'r', 'a', 'e', 'n'}
2   g   _r_a_an   {'m', 'i', 'r', 'a', 'e', 'n'}
1   b   _r_a_an   {'m', 'g', 'i', 'r', 'a', 'e', 'n'}
1   c   cr_a_an   {'m', 'g', 'b', 'i', 'r', 'a', 'e', 'n'}
1   o   croa_an   {'m', 'g', 'b', 'i', 'r', 'a', 'e', 'c', 'n'}
0   p   croa_an   {'m', 'g', 'b', 'i', 'r', 'a', 'e', 'c', 'n', 'o'}
0
bettertaught
6   e   _e__e_______   set()
6   r   _e__er______   {'e'}
6   t   _ettert____t   {'r', 'e'}
5   n   _ettert____t   {'r', 't', 'e'}
4   m   _ettert____t   {'r', 'n', 't', 'e'}
4   a   _etterta___t   {'m', 'r', 'e', 't', 'n'}
3   s   _etterta___t   {'m', 'r', 'a', 'e', 't', 'n'}
2   i   _etterta___t   {'m', 'r', 's', 'a', 'e', 't', 'n'}
1   l   _etterta___t   {'m', 'i', 'r', 's', 'a', 'e', 't', 'n'}
1   b   bet

...

1   p   nar_i_es   {'i', 'r', 's', 'd', 'a', 'e', 'c', 't', 'n', 'o'}
1   g   nargi_es   {'i', 'r', 's', 'd', 'a', 'p', 'e', 'c', 't', 'n', 'o'}
1   l   nargiles   {'g', 'i', 'r', 's', 'd', 'a', 'p', 'e', 'c', 't', 'n', 'o'}
1
phyllopodiform
6   i   _________i____   set()
5   t   _________i____   {'i'}
4   n   _________i____   {'i', 't'}
3   e   _________i____   {'i', 't', 'n'}
3   l   ___ll____i____   {'i', 't', 'n', 'e'}
2   s   ___ll____i____   {'i', 'e', 'l', 't', 'n'}
1   w   ___ll____i____   {'i', 's', 'e', 'l', 't', 'n'}
0   a   ___ll____i____   {'w', 'i', 's', 'e', 'l', 't', 'n'}
0
airgraphics
5   e   ___________   set()
5   i   _i______i__   {'e'}
4   n   _i______i__   {'i', 'e'}
4   s   _i______i_s   {'i', 'n', 'e'}
4   r   _ir_r___i_s   {'i', 's', 'n', 'e'}
4   a   air_ra__i_s   {'i', 'r', 's', 'e', 'n'}
4   g   airgra__i_s   {'i', 'r', 's', 'a', 'e', 'n'}
4   h   airgra_hi_s   {'g', 'i', 'r', 's', 'a', 'e', 'n'}
4   p   airgraphi_s   {'g', 'h', 'i', 'r', 's', 'a', 'e', 'n'}

4   r   variation   {'v', 'i', 's', 'a', 'e', 't', 'n', 'o'}
1
jake
6   a   _a__   set()
5   s   _a__   {'a'}
4   r   _a__   {'s', 'a'}
3   n   _a__   {'r', 's', 'a'}
2   t   _a__   {'r', 's', 'a', 'n'}
1   l   _a__   {'r', 's', 'a', 't', 'n'}
0   m   _a__   {'r', 's', 'a', 'l', 't', 'n'}
0
estivates
6   e   e______e_   set()
6   s   es_____es   {'e'}
6   t   est___tes   {'s', 'e'}
6   a   est__ates   {'t', 's', 'e'}
5   l   est__ates   {'t', 's', 'a', 'e'}
4   u   est__ates   {'s', 'a', 'e', 'l', 't'}
3   g   est__ates   {'u', 's', 'a', 'e', 'l', 't'}
2   r   est__ates   {'g', 'u', 's', 'a', 'e', 'l', 't'}
1   n   est__ates   {'g', 'u', 'r', 's', 'a', 'e', 'l', 't'}
0   o   est__ates   {'g', 'u', 'r', 's', 'a', 'e', 'l', 't', 'n'}
0
amphanthia
5   e   __________   set()
5   i   ________i_   {'e'}
4   o   ________i_   {'i', 'e'}
4   a   a___a___ia   {'i', 'o', 'e'}
4   n   a___an__ia   {'i', 'a', 'o', 'e'}
4   t   a___ant_ia   {'i', 'a', 'e', 'n', 'o'}
4   h   a__hanthia   {'i', 'a', '

2   g   _ri___n_   {'i', 'r', 'a', 'e', 't', 'n'}
1   p   _ri___n_   {'g', 'i', 'r', 'a', 'e', 't', 'n'}
1   f   fri___n_   {'g', 'i', 'r', 'p', 'a', 'e', 't', 'n'}
1   s   friss_ns   {'g', 'i', 'r', 'p', 'a', 'e', 'f', 't', 'n'}
1   o   frissons   {'g', 'i', 'r', 'p', 'a', 's', 'e', 'f', 't', 'n'}
1
eructed
6   e   e____e_   set()
5   s   e____e_   {'e'}
4   n   e____e_   {'s', 'e'}
4   d   e____ed   {'s', 'n', 'e'}
4   t   e___ted   {'s', 'n', 'd', 'e'}
3   x   e___ted   {'s', 'e', 't', 'n', 'd'}
3   c   e__cted   {'x', 's', 'e', 't', 'n', 'd'}
2   l   e__cted   {'x', 's', 'e', 'c', 't', 'n', 'd'}
1   a   e__cted   {'x', 's', 'e', 'c', 'l', 't', 'n', 'd'}
0   j   e__cted   {'x', 's', 'a', 'e', 'c', 'l', 't', 'n', 'd'}
0
titanaugite
6   e   __________e   set()
6   i   _i______i_e   {'e'}
6   t   tit_____ite   {'i', 'e'}
6   a   tita_a__ite   {'i', 't', 'e'}
6   n   titana__ite   {'i', 't', 'a', 'e'}
5   l   titana__ite   {'i', 'a', 'e', 't', 'n'}
4   r   titana__ite   {'i', 'a', 'e', 

5   s   _____e__   {'e'}
4   r   _____e__   {'s', 'e'}
3   t   _____e__   {'r', 's', 'e'}
3   a   ____ae_a   {'r', 's', 't', 'e'}
2   n   ____ae_a   {'r', 's', 'a', 'e', 't'}
2   i   _i__ae_a   {'r', 's', 'a', 'e', 't', 'n'}
2   l   _i__aela   {'i', 'r', 's', 'a', 'e', 't', 'n'}
2   c   _ic_aela   {'i', 'r', 's', 'a', 'e', 'l', 't', 'n'}
2   h   _ichaela   {'i', 'r', 's', 'a', 'e', 'c', 'l', 't', 'n'}
2   m   michaela   {'h', 'i', 'r', 's', 'a', 'e', 'c', 'l', 't', 'n'}
1
metaleptical
6   e   _e___e______   set()
5   r   _e___e______   {'e'}
5   i   _e___e__i___   {'r', 'e'}
5   l   _e__le__i__l   {'i', 'r', 'e'}
5   a   _e_ale__i_al   {'i', 'r', 'l', 'e'}
5   c   _e_ale__ical   {'i', 'r', 'a', 'e', 'l'}
5   t   _etale_tical   {'i', 'r', 'a', 'e', 'c', 'l'}
5   m   metale_tical   {'i', 'r', 'a', 'e', 'c', 'l', 't'}
5   p   metaleptical   {'m', 'i', 'r', 'a', 'e', 'c', 'l', 't'}
1
quadrantid
5   e   __________   set()
5   i   ________i_   {'e'}
4   o   ________i_   {'i', 'e'}
4   a   __

6   e   e______e_e_   set()
6   r   e______erer   {'e'}
6   t   e_____terer   {'r', 'e'}
6   n   en___nterer   {'r', 't', 'e'}
6   c   enc__nterer   {'r', 'n', 't', 'e'}
5   h   enc__nterer   {'r', 'e', 'c', 't', 'n'}
5   u   enc_unterer   {'h', 'r', 'e', 'c', 't', 'n'}
5   o   encounterer   {'u', 'h', 'r', 'e', 'c', 't', 'n'}
1
watchmanly
5   e   __________   set()
4   i   __________   {'e'}
4   a   _a____a___   {'i', 'e'}
3   r   _a____a___   {'i', 'a', 'e'}
3   l   _a____a_l_   {'i', 'r', 'a', 'e'}
2   b   _a____a_l_   {'i', 'r', 'a', 'e', 'l'}
2   c   _a_c__a_l_   {'b', 'i', 'r', 'a', 'e', 'l'}
1   u   _a_c__a_l_   {'b', 'i', 'r', 'a', 'e', 'c', 'l'}
1   h   _a_ch_a_l_   {'u', 'b', 'i', 'r', 'a', 'e', 'c', 'l'}
1   t   _atch_a_l_   {'u', 'h', 'b', 'i', 'r', 'a', 'e', 'c', 'l'}
1   m   _atchma_l_   {'u', 'h', 'b', 'i', 'r', 'a', 'e', 'c', 'l', 't'}
1   n   _atchmanl_   {'m', 'u', 'h', 'b', 'i', 'r', 'a', 'e', 'c', 'l', 't'}
1   y   _atchmanly   {'m', 'u', 'h', 'b', 'i', 'r', 'a', 'e